In [173]:
from datasets import Dataset, load_dataset
import requests
from openai import OpenAI
from pydantic import BaseModel
import tqdm
import time
from collections import Counter
from sklearn.cluster import DBSCAN
import numpy as np
import random

In [5]:
original_dataset = load_dataset('asadfgglie/BanBan_2024-10-17-Pure_text_raw_data')

In [6]:
url = 'https://free.v36.cm/v1/'
with open('api_key', 'r') as f:
    api_key = f.read()
# model = 'gemma-2-2b'
# model = 'llama3.1-8b'
model = 'gpt-4o-mini'
skip_index = -1
client = OpenAI(api_key=api_key, base_url=url)

In [7]:
class Expression(BaseModel):
    facial_expressions: list[str]

In [8]:
response = client.beta.chat.completions.parse(
    messages=[
        {
            'role':'system',
            'content':'你是一位專業的表情揣摩大師，你擅長根據已知的文字訊息揣摩使用者當下的表情。當使用者給你一句話時，你會將使用者的文字訊息與所對應的表情以json格式輸出。例如:\n'
            'user: 我不太確定你在說什麼...\n'
            "assistant: {'facial_expressions': ['困惑', '迷茫']}\n"
            'user: 我得說，朝歌，你錯過了一個億啊!\n'
            "assistant: {'facial_expressions': ['生氣', '失望', '憤怒']}\n"
            'user: *笑咪咪地看著朝歌* 朝歌你介意我在你的房間門口放幾個"掃地機器人"嗎?'
            "assistant: {'facial_expressions': ['微笑', '興奮']}\n"
            'user: 這真是讓我痛心疾首，太讓我難過了。\n'
            "assistant: {'facial_expressions': ['難過', '惋惜']}"
        },
        {
            'role':'user',
            'content':'哼!'
        }
    ],
    model=model,
    response_format=Expression
)
response.choices[0].message.parsed

Expression(facial_expressions=['不悅', '輕蔑', '生氣'])

In [9]:
def convert():
    for ex in tqdm.tqdm(original_dataset['train'], total=len(original_dataset['train'])):
        flag = True
        while flag:
            try:
                expression = client.beta.chat.completions.parse(
                    messages=[
                        {
                            'role':'system',
                            'content':'你是一位專業的表情揣摩大師，你擅長根據已知的文字訊息揣摩使用者當下的表情。當使用者給你一句話時，你會將使用者的文字訊息與所對應的表情以json格式輸出。例如:\n'
                            'user: 我不太確定你在說什麼...\n'
                            "assistant: {'facial_expressions': ['困惑', '迷茫']}\n"
                            'user: 我得說，朝歌，你錯過了一個億啊!\n'
                            "assistant: {'facial_expressions': ['生氣', '失望', '憤怒']}\n"
                            'user: *笑咪咪地看著朝歌* 朝歌你介意我在你的房間門口放幾個"掃地機器人"嗎?'
                            "assistant: {'facial_expressions': ['微笑', '興奮']}\n"
                            'user: 這真是讓我痛心疾首，太讓我難過了。\n'
                            "assistant: {'facial_expressions': ['難過', '惋惜']}"
                        },
                        {
                            'role':'user',
                            'content':ex['text']
                        }
                    ],
                    model=model,
                    response_format=Expression
                ).choices[0].message.parsed
                yield {
                    'sequences': ex['text'], 'candidate_labels': expression.facial_expressions
                }
                flag = False
            except Exception as e:
                print(f'{e.__class__.__name__}: {e}')
                time.sleep(5)
                flag = True

In [10]:
data = list(convert())

100%|██████████| 1488/1488 [1:14:22<00:00,  3.00s/it]


In [15]:
with open('tmp.txt', 'w', encoding='utf8') as f:
    for d in data:
        f.write(str(d) + '\n')

In [ ]:
_data = []
tmp = dict()
with open('tmp.txt', 'r', encoding='utf8') as f:
    for i, s in enumerate(f.readlines()):
        l = eval(s)
        assert len(l['candidate_labels']) > 0, i
        if l['sequences'] not in tmp:
            tmp[l['sequences']] = l['candidate_labels']
        else:
            tmp[l['sequences']] += l['candidate_labels']
for k,v in tmp.items():
    tmp[k] = list(set(v))
with open('tmp.txt', 'w', encoding='utf8') as f:
    for k,v in tmp.items():
        f.write(str({
            'sequences':k,'candidate_labels':v
        }) + '\n')
tmp = dict()
with open('tmp.txt', 'r', encoding='utf8') as f:
    for s in f.readlines():
        l = eval(s)
        _data.append(l)
        assert len(l['candidate_labels']) > 0
        for c in l['candidate_labels']:
            if c in tmp:
                tmp[c] += 1
            else:
                tmp[c] = 1
print(len(tmp))
print(sum(tmp.values()))
print(tmp.keys())
Counter(tmp).most_common()

In [159]:
session = requests.session()
for k in tmp:
    tmp[k] = {
        'counter': tmp[k],
        'embedding': session.post('http://26.171.83.203:12345/v1/embeddings', json={
            'input': f'這個表情是{k}。'
        }).json()['data'][0]['embedding']
    }

In [ ]:
dbscan = DBSCAN(0.25, min_samples=1)
dbscan.fit(np.array([d['embedding'] for d in tmp.values()]))
print(dbscan.labels_[np.argmax(dbscan.labels_)])
label_set: dict[int, set[str]] = dict()
for i, k in enumerate(tmp):
    if dbscan.labels_[i] not in label_set:
        label_set[dbscan.labels_[i]] = {k}
    else:
        label_set[dbscan.labels_[i]].add(k)
for v in label_set.values():
    print(v)

In [180]:
l = [
{'自然'}, {'自信', '認真', '思考'},

{'一本正經', '冷靜'}, {'挑眉'},

{'驚艷', '崇敬', '感激', '驚訝', '欣慰', '感嘆', '激動', '敬佩', '震驚',
 '驕傲', '滿足', '欣賞', '自豪', '讚嘆', '滿意', '尊敬', '熱情', '樂觀',
 '感慨', '微笑', '讚賞', '開心', '好奇', '興奮', '享受', '驚喜', '期待',
 '陶醉'},

{'理解', '同情', '安慰'}, {'冷笑'},

{'迷茫', '困惑', '懷疑', '猶豫'},

{'嘲諷', '失望', '厭煩', '自嘲', '鄙視', '感傷', '傷心', '難過', '懊惱',
 '抗拒', '尷尬', '嫌棄', '沮喪', '苦笑', '委屈', '惋惜', '厭惡', '嘲笑',
 '不安', '擔心', '防備', '敬畏', '警惕', '不滿', '不耐煩', '憤怒',
 '抗議', '挑釁'},

{'不在意', '不屑'}, {'無奈'},

{'害羞'}, {'渴望'},

{'急切', '焦急', '緊張'},

{'神秘', '狡黠'},

{'強調', '叮囑'},

{'嚴肅', '虔誠'}, {'懶散'}
]

In [181]:
for d in _data:
    c_set = set(d['candidate_labels'])
    d['not_candidate_labels'] = []
    index = list(range(len(l)))
    random.shuffle(index)
    num = 0
    for i in index:
        if num >= len(c_set):
            break
        if len(c_set.union(d['not_candidate_labels']).intersection(l[i])) == 0:
            d['not_candidate_labels'].append(random.choice(list(l[i])))
            num += 1
_data

[{'sequences': '就像圖片裡面說的，這是人類最新的掃"人"機器人，專門用來清理半徑20公尺以內、讓人反感的傢伙。',
  'candidate_labels': ['自然'],
  'not_candidate_labels': ['害羞']},
 {'sequences': '只是在使用時記得先詳閱使用手冊，注意哪面朝上、哪面朝下，否則你也會一起變得乾乾淨淨喔！',
  'candidate_labels': ['認真'],
  'not_candidate_labels': ['懶散']},
 {'sequences': '*笑咪咪地看著朝歌* 朝歌你介意我在你的房間門口放幾個"掃地機器人"嗎?',
  'candidate_labels': ['微笑', '開心'],
  'not_candidate_labels': ['害羞', '嚴肅']},
 {'sequences': '我敢保證，你的房間也會變得乾乾淨淨喔!',
  'candidate_labels': ['開心'],
  'not_candidate_labels': ['懶散']},
 {'sequences': '不需要"掃地機器人"嗎?',
  'candidate_labels': ['困惑', '好奇'],
  'not_candidate_labels': ['懶散', '害羞']},
 {'sequences': '我得說，朝歌，你錯過了一個億啊!',
  'candidate_labels': ['失望', '惋惜'],
  'not_candidate_labels': ['叮囑', '冷笑']},
 {'sequences': '這真是讓我痛心疾首，太讓我難過了。',
  'candidate_labels': ['難過', '惋惜'],
  'not_candidate_labels': ['害羞', '懶散']},
 {'sequences': '你看看這可愛的小機器人，黑色圓滾滾的身軀，頭頂可愛的小拉環，你不覺得她很可愛嗎?',
  'candidate_labels': ['興奮'],
  'not_candidate_labels': ['理解']},
 {'sequences': '朝歌，我可以養她嗎?',
  'candidate_labels':

In [182]:
data = _data

In [183]:
Dataset.from_list(data).push_to_hub('BanBan_2024-10-17-facial_expressions')

Deleting unused files from dataset repository: 100%|██████████| 1/1 [00:00<00:00,  2.54it/s]
